In [7]:
from unicodedata import category
import torch
from tqdm.notebook import tqdm
import numpy as np 
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import f1_score
import random
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import json

In [9]:
help(tqdm)

Help on class tqdm_notebook in module tqdm.notebook:

class tqdm_notebook(tqdm.std.tqdm)
 |  tqdm_notebook(*args, **kwargs)
 |  
 |  Experimental IPython/Jupyter Notebook widget using tqdm!
 |  
 |  Method resolution order:
 |      tqdm_notebook
 |      tqdm.std.tqdm
 |      tqdm.utils.Comparable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Supports the usual `tqdm.tqdm` parameters as well as those listed below.
 |      
 |      Parameters
 |      ----------
 |      display  : Whether to call `display(self.container)` immediately
 |          [default: True].
 |  
 |  __iter__(self)
 |      Backward-compatibility to use: for x in tqdm(iterable)
 |  
 |  clear(self, *_, **__)
 |      Clear current bar display.
 |  
 |  close(self)
 |      Cleanup and (if leave=False) close the progressbar.
 |  
 |  display(self, msg=None, pos=None, close=False, bar_style=None, check_delay=True)
 |      Use `self.sp` to display `msg` in the specif

In [2]:
#getting unique categores this will be the first function
def pretraining(df,companyname,instanceid):
    possible_labels = df.category.unique()
    label_dict = {}
    for index, possible_label in enumerate(possible_labels):
        label_dict[possible_label] = index
    category_dict = {}
    count = 0
    for category in label_dict.keys():
        category_dict[count] = category
        count = count + 1
    category_file_name = "category" + companyname + str(instanceid) + ".txt"
    text_file = open(category_file_name, "w")
    categories = json.dumps(category_dict)
    text_file.write(categories)
    text_file.close()
    return category_dict


def training(df,companyname,instanceid):
    possible_labels = df.category.unique()
    label_dict = {}
    for index, possible_label in enumerate(possible_labels):
        label_dict[possible_label] = index
    df.category = df['category'].map(label_dict)
    X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                    df.category.values, 
                                                    test_size=0.15, 
                                                    random_state=42,
                                                    stratify=df.category.values)
    df['data_type'] = ['not_set']*df.shape[0]
    df.loc[X_train, 'data_type'] = 'train'
    df.loc[X_val, 'data_type'] = 'val'
    df.groupby(['category', 'data_type']).count()

    tokenizer = BertTokenizer.from_pretrained(
        'bert-base-uncased',
        do_lower_case=True
    )
    print(df)
    print("df[df.data_type=='train'].question.values",type(df[df.data_type=='train'].question.values))
    
    encoded_data_train = tokenizer.batch_encode_plus(
        
        df[df.data_type=='train'].question.values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )

    encoded_data_val = tokenizer.batch_encode_plus(
        df[df.data_type=='val'].question.values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )

    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    labels_train = torch.tensor(df[df.data_type=='train'].category.values)

    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    labels_val = torch.tensor(df[df.data_type=='val'].category.values)

    dataset_train = TensorDataset(input_ids_train, 
                                attention_masks_train,
                                labels_train)

    dataset_val = TensorDataset(input_ids_val, 
                                attention_masks_val,
                            labels_val)
    model = BertForSequenceClassification.from_pretrained(
                                        'bert-base-uncased', 
                                        num_labels = len(label_dict),
                                        output_attentions = False,
                                        output_hidden_states = False
                                        )
    batch_size = 4

    dataloader_train = DataLoader(
        dataset_train,
        sampler=RandomSampler(dataset_train),
        batch_size=batch_size
    )

    dataloader_val = DataLoader(
        dataset_val,
        sampler=RandomSampler(dataset_val),
        batch_size=32
    )
    optimizer = AdamW(
        model.parameters(),
        lr = 1e-5,
        eps = 1e-8
    )
    epochs = 1

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps = len(dataloader_train)*epochs
    )
    def f1_score_func(preds, labels):
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return f1_score(labels_flat, preds_flat, average = 'weighted')

    def accuracy_per_class(preds, labels):
        label_dict_inverse = {v: k for k, v in label_dict.items()}
        
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        
        for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat==label]
            y_true = labels_flat[labels_flat==label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')
            

    seed_val = 17
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    def evaluate(dataloader_val):

        model.eval()
        
        loss_val_total = 0
        predictions, true_vals = [], []
        
        for batch in tqdm(dataloader_val):
            
            batch = tuple(b.to(device) for b in batch)
            
            inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'labels':         batch[2],
                    }

            with torch.no_grad():        
                outputs = model(**inputs)
                
            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)
        
        loss_val_avg = loss_val_total/len(dataloader_val) 
        
        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)
                
        return loss_val_avg, predictions, true_vals
    for epoch in tqdm(range(1, epochs+1)):
        model.train()
        loss_train_total = 0
        
        progress_bar = tqdm(dataloader_train, 
                            desc='Epoch {:1d}'.format(epoch), 
                            leave=False, 
                            disable=False)
        
        for batch in progress_bar:
            model.zero_grad()
            batch = tuple(b.to(device) for b in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': batch[2]
            }
            
            outputs = model(**inputs)
            loss = outputs[0]
            loss_train_total +=loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
        
        torch.save(model.state_dict(), f'BERT_ft_Epoch{epoch}.model')
        
        tqdm.write('\nEpoch {epoch}')
        
        loss_train_avg = loss_train_total/len(dataloader_train)
        tqdm.write(f'Training loss: {loss_train_avg}')
        
        val_loss, predictions, true_vals = evaluate(dataloader_val)
        val_f1 = f1_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (weighted): {val_f1}')
    Bert_model_Name = "BertModel" + companyname + str(instanceid) 
    torch.save(model,Bert_model_Name)




In [3]:
data = pd.read_csv("asktalos_dataset.csv", encoding='cp1252')


In [4]:
data.drop(['Unnamed: 0'],axis = 1,inplace = True)

In [5]:
data

,question,category
0,difference between machine learning and artifi...,course
1,what is the difference between machine learnin...,course
2,what are the job prospects for data science fo...,course
3,what do i get if i subscribe to this certificate?,general
4,what are the things i should i expect from thi...,general
...,...,...
506,greetings,greetings
507,good morning,greetings
508,gm,greetings
509,hello,greetings


In [6]:
companyname = "Amazon"
instanceid = 2
categories = pretraining(data,companyname,instanceid) 
training = training(data,companyname,instanceid)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


                                              question  category data_type
0    difference between machine learning and artifi...         0     train
1    what is the difference between machine learnin...         0       val
2    what are the job prospects for data science fo...         0     train
3    what do i get if i subscribe to this certificate?         1     train
4    what are the things i should i expect from thi...         1     train
..                                                 ...       ...       ...
506                                          greetings         6     train
507                                       good morning         6     train
508                                                 gm         6     train
509                                              hello         6     train
510                                           morning!         6       val

[511 rows x 3 columns]
df[df.data_type=='train'].question.values <class 'numpy.ndarray'>


C:\Users\thewa\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenc

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/109 [00:00<?, ?it/s]

KeyboardInterrupt: 